# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [38]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
import math
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'aapl'
api_url =f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': '/AGATCSEBL ND(EKE) AQRS ATGSOLMLN',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'openSource': 'icafolif',
 'close': None,
 'closeTime': None,
 'closeSource': 'afloiifc',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 140.2,
 'latestSource': 'IEX real time price',
 'latestTime': '2:19:09 PM',
 'latestUpdate': 1619095578076,
 'latestVolume': None,
 'iexRealtimePrice': 144.48,
 'iexRealtimeSize': 105,
 'iexLastUpdated': 1686053780463,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 149.15,
 'previousVolume': 145247306,
 'change': -2.8,
 'changePercent': -0.02004,
 'volume': None,
 'iexMarketPercent': 0.0210044348396989,
 'iexVolume': 20045

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio =data['peRatio']
pe_ratio

43.47

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns  =my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['peRatio'],
            'N/A'
        ],
            index = my_columns
        ),
            ignore_index =True
        )
final_dataframe
#        print(symbol)

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,122.590,54.31,N/A
1,AAL,18.255,-1.29,N/A
2,AAP,153.010,23.09,N/A
3,AAPL,139.770,43.94,N/A
4,ABBV,107.330,23.45,N/A
...,...,...,...,...
500,YUM,107.680,30.42,N/A
501,ZBH,164.505,-224.42,N/A
502,ZBRA,410.374,45.78,N/A
503,ZION,45.590,19.18,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [15]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace= True)
final_dataframe.drop('index', axis =1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,42.235,2.66,473
1,NLOK,21.590,4.23,926
2,AIV,4.988,4.75,4009
3,BIO,593.110,5.03,33
4,UNM,24.660,5.31,811
5,AFL,46.960,7.53,425
6,ALL,114.650,7.83,174
7,EBAY,58.811,8.1,340
8,MET,50.670,8.23,394
9,CPB,51.029,8.68,391


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [16]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [18]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [19]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,42.235,2.66,473
1,NLOK,21.590,4.23,926
2,AIV,4.988,4.75,4009
3,BIO,593.110,5.03,33
4,UNM,24.660,5.31,811
5,AFL,46.960,7.53,425
6,ALL,114.650,7.83,174
7,EBAY,58.811,8.1,340
8,MET,50.670,8.23,394
9,CPB,51.029,8.68,391


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [20]:
symbol = 'AAPL'
batch_api_call_url =f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
#data.status_code

#Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

#Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']   

#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value =data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']  
ev_to_ebitda = enterprise_value/ebitda

#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [21]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )
        

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [23]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,216.450,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,236.890,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,34.320,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.320,N/A,14.27,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,54.730,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,30.778,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,153.290,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1926.250,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.260,N/A,31.86,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [25]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [26]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [28]:
from scipy.stats import percentileofscore as score 
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.81,N/A,54.85,0.867327,7.77,0.738614,7.0000,0.807921,33.147053,0.847525,13.617447,0.821782,N/A
1,AAL,18.23,N/A,-1.32,0.150495,-1.89,0.0475248,0.4142,0.029703,-6.336785,0.029703,1.449796,0.0574257,N/A
2,AAP,154.20,N/A,22.27,0.455446,2.72,0.388119,1.0600,0.142574,9.821859,0.190099,2.305407,0.110891,N/A
3,AAPL,142.49,N/A,43.94,0.80198,37.56,0.966337,9.0000,0.871287,31.969806,0.841584,22.744228,0.964356,N/A
4,ABBV,108.06,N/A,23.55,0.475248,12.21,0.845545,4.6000,0.677228,15.015946,0.449505,9.192823,0.633663,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.68,N/A,29.94,0.623762,-3.95,0.0435644,5.5000,0.730693,22.594316,0.70099,9.768963,0.669307,N/A
501,ZBH,165.19,N/A,-223.72,0.0257426,2.76,0.390099,4.5900,0.675248,25.362451,0.766337,7.791955,0.506931,N/A
502,ZBRA,408.66,N/A,46.11,0.817822,10.92,0.833663,5.0000,0.708911,30.758564,0.831683,11.233166,0.720792,N/A
503,ZION,45.42,N/A,19.05,0.39802,1.03,0.10297,2.4200,0.391089,10.717839,0.241584,2.809515,0.140594,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [30]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.81,N/A,54.85,0.867327,7.77,0.738614,7.0000,0.807921,33.147053,0.847525,13.617447,0.821782,0.816634
1,AAL,18.23,N/A,-1.32,0.150495,-1.89,0.0475248,0.4142,0.029703,-6.336785,0.029703,1.449796,0.0574257,0.0629703
2,AAP,154.20,N/A,22.27,0.455446,2.72,0.388119,1.0600,0.142574,9.821859,0.190099,2.305407,0.110891,0.257426
3,AAPL,142.49,N/A,43.94,0.80198,37.56,0.966337,9.0000,0.871287,31.969806,0.841584,22.744228,0.964356,0.889109
4,ABBV,108.06,N/A,23.55,0.475248,12.21,0.845545,4.6000,0.677228,15.015946,0.449505,9.192823,0.633663,0.616238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.68,N/A,29.94,0.623762,-3.95,0.0435644,5.5000,0.730693,22.594316,0.70099,9.768963,0.669307,0.553663
501,ZBH,165.19,N/A,-223.72,0.0257426,2.76,0.390099,4.5900,0.675248,25.362451,0.766337,7.791955,0.506931,0.472871
502,ZBRA,408.66,N/A,46.11,0.817822,10.92,0.833663,5.0000,0.708911,30.758564,0.831683,11.233166,0.720792,0.782574
503,ZION,45.42,N/A,19.05,0.39802,1.03,0.10297,2.4200,0.391089,10.717839,0.241584,2.809515,0.140594,0.254851


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [31]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [32]:
portfolio_input()

Enter the value of your portfolio:10000000


In [39]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,40.335,4958,-7.11,0.112871,0.5165,0.0574257,0.7070,0.0653465,4.257467,0.0415842,0.696551,0.0138614,0.0582178
1,AAL,18.230,10970,-1.32,0.150495,-1.8900,0.0475248,0.4142,0.029703,-6.336785,0.029703,1.449796,0.0574257,0.0629703
2,UNM,25.385,7878,5.45,0.19604,0.4707,0.0534653,0.4351,0.0316832,2.668336,0.0356436,0.411313,0.0039604,0.0641584
3,F,10.949,18266,-265.59,0.0237624,1.3200,0.165347,0.3337,0.0148515,4.499710,0.0455446,2.267374,0.106931,0.0712871
4,MET,51.740,3865,8.24,0.20396,0.6196,0.0613861,0.6850,0.0594059,4.665472,0.0475248,0.660561,0.0118812,0.0768317
5,BA,203.670,981,-25.47,0.0752475,-9.7300,0.0336634,1.9000,0.316832,-26.982518,0.0178218,-190.524093,0.0019802,0.0891089
6,FTI,11.010,18165,-0.86,0.156436,1.2500,0.143564,0.3919,0.0217822,3.436059,0.0376238,2.264161,0.10495,0.0928713
7,HIG,49.710,4023,10.49,0.223762,1.0190,0.0990099,0.8611,0.0891089,4.253455,0.039604,0.829328,0.0217822,0.0946535
8,ALL,112.650,1775,8.05,0.2,1.3800,0.176238,0.7740,0.0752475,2.475696,0.0336634,0.759239,0.0158416,0.100198
9,LNC,49.690,4024,12.40,0.255446,0.4372,0.0514851,0.5264,0.0415842,8.606255,0.146535,0.505388,0.00792079,0.100594


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [34]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [35]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [36]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [37]:
writer.save()